# 🧠 Clasificación, Limpieza y Evaluación con ID por Ciudad

## 📂 Cargar base de datos desde ruta verificada

In [6]:
import pandas as pd

# Cargar el dataset
df_raw = pd.read_csv('../../datos/dataset_ejemplo_1300.csv')

# Crear variable objetivo: Alta conectividad
df_raw["Alta_conectividad"] = (df_raw["Horas_Internet"] > 3.5).astype(int)

df_raw.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../datos/dataset_ejemplo_1300.csv'

In [2]:
import os

print("📂 Directorio actual:", os.getcwd())
print("📁 Contenido actual:", os.listdir())
print("📁 Contenido de carpeta 'datos':", os.listdir('./datos') if os.path.exists('./datos') else "❌ No existe carpeta 'datos'")


📂 Directorio actual: D:\iCloudDrive\analisis-predictivo-gestion-datos\Material complementario
📁 Contenido actual: ['.ipynb_checkpoints', 'Metricas_Evaluacion_Clasificacion.pdf', 'Notebook_FINAL_ID_Ciudad.ipynb', 'Odds_Logit_Interpretacion.pdf', 'Sesion 2 _ampliada.ipynb', 'Sesion0_Estadistica_Basica.ipynb', 'Sesion0_Estadistica_Basica.pdf']
📁 Contenido de carpeta 'datos': ❌ No existe carpeta 'datos'


## 🎯 Crear variables `Alta_conectividad` y `Nivel_conectividad`

In [3]:
df_raw['Alta_conectividad'] = (df_raw['Horas_Internet'] > 3.5).astype(int)

def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw['Nivel_conectividad'] = df_raw['Horas_Internet'].apply(clasificar_conectividad)

NameError: name 'df_raw' is not defined

## 🏙️ Crear identificador único para cada ciudad

In [ ]:
df_raw['ID_Ciudad'] = df_raw['Ciudad'].astype('category').cat.codes
df_raw[['Ciudad', 'ID_Ciudad']].drop_duplicates().sort_values('ID_Ciudad')

## 🧼 Imputación de valores faltantes

In [ ]:
from sklearn.impute import SimpleImputer

df_clean = df_raw.copy()
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])
imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

## 🔣 Codificación de variables categóricas

In [ ]:
df_clean = pd.get_dummies(df_clean, drop_first=True)

## 📏 Escalado de columnas numéricas

In [ ]:
from sklearn.preprocessing import MinMaxScaler
cols_numericas = df_clean.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

## ✅ Filtrar solo columnas numéricas para el modelo

In [ ]:
Xc = df_clean.drop(columns=['Alta_conectividad'])
Xc = Xc.select_dtypes(include=['int64', 'float64'])
yc = df_clean['Alta_conectividad']

## 🤖 Entrenar modelo y calcular exactitud

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(Xc_train, yc_train)
yc_pred = model.predict(Xc_test)
acc = accuracy_score(yc_test, yc_pred)
print('✅ Exactitud:', round(acc, 4))

## 📊 Evaluación con métricas adicionales

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print("📌 Matriz de confusión:")
print(confusion_matrix(yc_test, yc_pred))

print("\n📌 Reporte de clasificación:")
print(classification_report(yc_test, yc_pred))

print("\n📌 Distribución de clases en y_test:")
print(yc_test.value_counts(normalize=True))

## 🧪 Validación cruzada

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(LogisticRegression(max_iter=200), Xc, yc, cv=5)
print("📈 Scores validación cruzada:", scores)
print("Promedio:", scores.mean())